In [3]:
!pip install opencv-python

     ---------------------------------------- 35.6/35.6 MB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 14.7/14.7 MB 1.4 MB/s eta 0:00:00


In [4]:
# import libraries
import os
import subprocess
import shutil
import fnmatch
from osgeo import gdal
import cv2
from PIL import Image
import numpy as np


In [5]:
# function that splits Geotiffs into required pieces

def create_tiles(src_path, dst_path):

    # specify tile dimension
    tile_size_x = 512
    tile_size_y = 512

    # dynamically get the image dimension
    dataset = gdal.Open(src_path)
    band = dataset.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize

    # clip image using using tile_size and gdal_translate iteratively
    for i in range(0, xsize, tile_size_x):
        for j in range(0, ysize, tile_size_y):
            com_string = "gdal_translate -of GTIFF -srcwin " + str(i) + ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(src_path) + " " + str(dst_path) + "_" + str(i) + "_" + str(j) + ".tif"
            print(com_string)
            # os.system(com_string)

In [12]:
import glob
import pandas as pd
import torch

#from torch.utils.data import Dataset

ModuleNotFoundError: No module named 'torch'

In [11]:
DATA_PATH = 'data'
data_map = []
pla_map = []
sen_map = []

for tile in glob.glob(f'{DATA_PATH}/Tile *'):

  for planet_img in glob.glob(f'{tile}/Planet_*_FullScene/*.tif'):
    pla_map.append(planet_img)
  
  for sentinel_img in glob.glob(f'{tile}/Sentinel_*/*.tif'):
     sen_map.append(sentinel_img)

df_p = pd.DataFrame({'img_path': pla_map[::2]})


In [ ]:
pla_pth = "" # fill in path to results empty dir. should end n '/'
sen_pth = r"" # fill in path to results empty dir

In [ ]:
def gtiff_split(map, out_pth):
  for i in map: 
    tiff_name = i
    new_dir = tiff_name.split('\\')[-1].replace('.tif', '')
    new_path = os.path.join(out_path, new_dir) #out_path/some_name
    create_tiles(i, new_path)

In [ ]:
gtiff_split(pla_map, pla_pth)
gtiff_split(sen_map, sen_pth)

In [ ]:
# function converts geotiff to png files (8bit)
def geotiff_to_png(input_tiff_img, output_png_img, output_pix_type='Byte', output_format='png', compress='png', percentiles=[2, 98]):

    img_src = gdal.Open(input_tiff_img)

    cmd = ['gdal_translate', '-ot', output_pix_type, '-of', output_format, '-co', compress]

    for band_id in range(img_src.RasterCount):
        band_id = band_id + 1
        band = img_src.GetRasterBand(band_id)

        # cal the band minimum and max values of raster
        b_min = band.GetMinimum()
        b_max = band.GetMaximum()

        # if not exist minimum and maximum values
        if b_min is None or b_max is None:
            (b_min, b_max) = band.ComputeRasterMinMax(1)
            if b_min == b_max:
                b_min -= 1

        cmd.append('-scale_{}'.format(band_id))
        cmd.append('{}'.format(b_min))
        cmd.append('{}'.format(b_max))
        cmd.append('{}'.format(0))
        cmd.append('{}'.format(255))

    cmd.append(input_tiff_img)
    cmd.append(output_png_img)
    print("Conversion command:", cmd)
    subprocess.call(cmd)

In [ ]:
# function that activates the converter wrt geotiff in input path
def converter(input_path, output_path):
    # convert each file to output
    for file in os.listdir(input_path):
        geotiff_to_png(os.path.join(input_path, file), (output_path + file[:-4] + "png"))
    print(f"success! all files have been converted to png")

In [ ]:
# move xml extension files from png_path to its xml_path
def separate_png_xml(src_path, dst_path, ext):
    for file in fnmatch.filter(os.listdir(src_path), ext):
        shutil.move(os.path.join(src_path, file), os.path.join(dst_path, file))
    print(f"success!! xml file has been moved to Palm_xml")

In [ ]:
# directories
geotiff_path = "/content/drive/MyDrive/Split_Geotiffs/20170929_145600-01_Mosaic_clip/"
png_path = "/content/drive/MyDrive/images/"
xml_path = "/content/drive/MyDrive/xml/"

In [ ]:
# Convert the geotiffs to png
converter(input_path=geotiff_path, output_path=png_path)

Conversion command: ['gdal_translate', '-ot', 'Byte', '-of', 'png', '-co', 'png', '-scale_1', '37.0', '453.0', '0', '255', '-scale_2', '140.0', '629.0', '0', '255', '-scale_3', '109.0', '572.0', '0', '255', '-scale_4', '893.0', '5871.0', '0', '255', '/content/drive/MyDrive/Split_Geotiffs/20170929_145600-01_Mosaic_clip/20170929_145600-01_Mosaic_clip_0_0.tif', '/content/drive/MyDrive/images/20170929_145600-01_Mosaic_clip_0_0png']
Conversion command: ['gdal_translate', '-ot', 'Byte', '-of', 'png', '-co', 'png', '-scale_1', '31.0', '405.0', '0', '255', '-scale_2', '139.0', '517.0', '0', '255', '-scale_3', '118.0', '562.0', '0', '255', '-scale_4', '1204.0', '4412.0', '0', '255', '/content/drive/MyDrive/Split_Geotiffs/20170929_145600-01_Mosaic_clip/20170929_145600-01_Mosaic_clip_0_512.tif', '/content/drive/MyDrive/images/20170929_145600-01_Mosaic_clip_0_512png']
Conversion command: ['gdal_translate', '-ot', 'Byte', '-of', 'png', '-co', 'png', '-scale_1', '29.0', '562.0', '0', '255', '-scale_

In [ ]:
# Remove the all xml files from the png directory to xml folder
separate_png_xml(src_path=png_path, dst_path=xml_path, ext='*.xml')

success!! xml file has been moved to Palm_xml


In [ ]:
# find the number of images channels in a particular image
image = '/content/drive/MyDrive/images/Kudos1.png'

img = Image.open(image)

im1 = Image.Image.getbands(img)

print(im1)

# convert to numpy array
img = np.array(img)

# find number of channels
if img.ndim == 2:
    channels = 1
    print("image has 1 channel")
else:
    channels = img.shape[-1]
    print("image has", channels, "channels")

('R', 'G', 'B')
image has 3 channels


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive
